# Limpieza de caracteres no deseados

In [5]:
import pandas as pd
import re

# Función para eliminar caracteres no deseados
def clean_text(text):
    # Eliminar puntuación, números y caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Eliminar espacios extra
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 1. Cargar el CSV original
df = pd.read_csv('no_stopwords_abstracts.csv')

# 2. Limpiar los abstracts
df['Lemmas'] = df['Lemmas'].apply(clean_text)

# 3. Guardar el resultado en un nuevo CSV
df.to_csv('cleared_abstracts.csv', index=False)

print("Limpieza completada. Archivo guardado como 'cleared_abstracts.csv'.")


Limpieza completada. Archivo guardado como 'cleared_abstracts.csv'.


# LDA implementado por temas y Search Grid

In [6]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from sklearn.model_selection import ParameterGrid
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# 1. Cargar los datos
df = pd.read_csv('cleared_abstracts.csv')

# 2. Filtrar abstracts por tema
topics = df['Topic'].unique()
results = {}

def preprocess_by_topic(df, topic):
    # Filtrar el dataframe por un tema específico
    filtered_df = df[df['Topic'] == topic]
    
    # Dividir los abstracts en listas de palabras (tokenización)
    corpus = [abstract.split() for abstract in filtered_df['Lemmas']]
    
    # Crear diccionario y corpus para Gensim
    id2word = corpora.Dictionary(corpus)
    corpus_gensim = [id2word.doc2bow(text) for text in corpus]
    
    return corpus_gensim, id2word, corpus

# 3. Definir función para entrenar LDA
def train_lda(corpus, id2word, num_topics, iterations):
    lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, iterations=iterations, random_state=100)
    return lda_model

# 4. Definir el espacio de parámetros (grid search)
param_grid = {
    'num_topics': [8, 10, 12, 15],
    'iterations': [100, 200, 300]
}
grid = ParameterGrid(param_grid)

# 5. Entrenar y evaluar modelos LDA por cada tema
for topic in topics:
    print(f'Procesando tema: {topic}')
    
    # Preprocesar el corpus para este tema
    corpus_gensim, id2word, corpus = preprocess_by_topic(df, topic)
    
    best_model = None
    best_score = float('-inf')
    best_params = None

    # Grid search
    for params in grid:
        num_topics = params['num_topics']
        iterations = params['iterations']
        
        # Entrenar el modelo
        lda_model = train_lda(corpus_gensim, id2word, num_topics=num_topics, iterations=iterations)
        
        # Evaluar coherencia
        coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=id2word, coherence='c_v')
        coherence_score = coherence_model_lda.get_coherence()
        
        # Guardar el mejor modelo
        if coherence_score > best_score:
            best_model = lda_model
            best_score = coherence_score
            best_params = params

    # Guardar resultados del mejor modelo por tema
    results[topic] = {
        'model': best_model,
        'coherence_score': best_score,
        'params': best_params
    }

    # Visualizar los temas del mejor modelo con pyLDAvis
    lda_vis = gensimvis.prepare(best_model, corpus_gensim, id2word)
    pyLDAvis.save_html(lda_vis, f'lda_visualization_{topic}.html')
    print(f'Mejor modelo para {topic} con coherencia {best_score}, número de temas: {best_params["num_topics"]}, iteraciones: {best_params["iterations"]}')
    
# Finalmente puedes explorar cada archivo HTML para visualizar los temas.


Procesando tema: SustainableFinance
Mejor modelo para SustainableFinance con coherencia 0.3457284999605327, número de temas: 15, iteraciones: 200
Procesando tema: CorporateFinance
Mejor modelo para CorporateFinance con coherencia 0.33862786797328764, número de temas: 12, iteraciones: 300
Procesando tema: AssetPricingModels
Mejor modelo para AssetPricingModels con coherencia 0.3951748524351609, número de temas: 15, iteraciones: 300
Procesando tema: BehavioralFinance
Mejor modelo para BehavioralFinance con coherencia 0.33789321567307085, número de temas: 10, iteraciones: 300
Procesando tema: FinancialDerivatives
Mejor modelo para FinancialDerivatives con coherencia 0.3797934305935555, número de temas: 8, iteraciones: 100
Procesando tema: FinancialEngineering
Mejor modelo para FinancialEngineering con coherencia 0.37578961574716774, número de temas: 15, iteraciones: 100
Procesando tema: FinancialRisk
Mejor modelo para FinancialRisk con coherencia 0.48156332718993217, número de temas: 12, 